In [1]:
import duckdb

In [2]:
# create a connection to a new DuckDB database file
conn = duckdb.connect('my_duckdb_database.db')

In [3]:
conn2 = duckdb.connect(':memory:')

In [3]:
# create a table
conn.execute('''
  CREATE TABLE employees (
      id INTEGER PRIMARY KEY,
      name VARCHAR,
      age INTEGER,
      department VARCHAR
  )'''
)

CatalogException: Catalog Error: Table with name "employees" already exists!

In [4]:
conn.execute('SHOW TABLES').df()

,name
0,employees


In [19]:
# insert data into the table
conn.execute('''
    INSERT INTO employees VALUES
      (1, 'Alice', 30, 'HR'),
      (2, 'Bob', 35, 'Engineering'),
      (3, 'Charlie', 28, 'Marketing'),
      (4, 'David', 40, 'Engineering')
   '''
)

In [5]:
conn.execute('''
    SELECT * from employees
'''
).df()

,id,name,age,department
0,1,Alice,30,HR
1,2,Bob,35,Engineering
2,3,Charlie,28,Marketing
3,4,David,40,Engineering


In [6]:
conn.execute('''
  SELECT 
    department,
    COUNT(*) AS employee_count,
  FROM 
    employees
  GROUP BY 
    department
'''    
).df()

,department,employee_count
0,HR,1
1,Marketing,1
2,Engineering,2


In [18]:
conn.execute(
    '''
    TRUNCATE employees
    '''
)

In [7]:
conn.execute('''
  SELECT 
    AVG(age) AS average_age
  FROM employees
'''
).df()

,average_age
0,33.25


In [8]:
conn.execute('''
  SELECT
    department,
    MAX(age) AS oldest_age
  FROM
    employees
  GROUP BY
    department
''').df()

,department,oldest_age
0,HR,30
1,Engineering,40
2,Marketing,28


In [10]:
conn.execute('''
  SELECT
    department,
    AVG(age) AS average_age
  FROM 
    employees
  GROUP BY
    department
''').df()

,department,average_age
0,HR,30.0
1,Marketing,28.0
2,Engineering,37.5


In [11]:
# create an in-memory copy of the database
conn = duckdb.connect()

In [12]:
# create first table - orders
conn.execute(
    '''
    CREATE TABLE orders (
      order_id INTEGER,
      customer_id INTEGER,
      amount FLOAT
    )
    '''
)

In [13]:
# add some records to the orders table
conn.execute(
    '''
    INSERT INTO orders
    VALUES (1, 1, 100.0),
           (2, 2, 200.0),
           (3, 1, 150.0)
    '''       
)

In [14]:
# create second table - customers
conn.execute(
    '''
    CREATE TABLE customers (
      customer_id INTEGER,
      name VARCHAR
    )
    '''
)

In [15]:
conn.execute(
    '''
    INSERT INTO customers
    VALUES (1, 'Alice'),
           (2, 'Bob')
    '''       
)

In [16]:
display(conn.execute(
    '''
    SELECT * FROM orders
    '''
    ).df()
)

display(conn.execute(
    '''
    SELECT * from customers
    '''
    ).df()
)

,order_id,customer_id,amount
0,1,1,100.0
1,2,2,200.0
2,3,1,150.0


,customer_id,name
0,1,Alice
1,2,Bob


In [17]:
# join the two tables
conn.execute(
    '''
    SELECT
      customers.customer_id,
      customers.name,
      orders.amount,
    FROM
      orders
    JOIN
      customers
    ON
      orders.customer_id = customers.customer_id
    ORDER by
      customers.customer_id
    '''
).df()

,customer_id,name,amount
0,1,Alice,100.0
1,1,Alice,150.0
2,2,Bob,200.0
